In [94]:
import sys
import re
import torch
import torchtext
import os
import random
from pprint import pprint
from collections import Counter

sys.path.insert(0, '../preprocess/')
sys.path.insert(0, '../../coarse2fine.git/src')

from sketch_generation import Sketch
from tree import SketchRepresentation
import table

In [45]:
UNK_WORD = '<unk>'
UNK = 0
PAD_WORD = '<blank>'
PAD = 1
BOS_WORD = '<s>'
BOS = 2
EOS_WORD = '</s>'
EOS = 3
SKP_WORD = '<sk>'
SKP = 4
RIG_WORD = '<]>'
RIG = 5
LFT_WORD = '<[>'
LFT = 6
SPECIAL_TOKEN_LIST = [UNK_WORD, PAD_WORD, BOS_WORD, EOS_WORD, SKP_WORD, RIG_WORD, LFT_WORD]

## Explore TableDataset fields

In [7]:
def print_field(data, field):
    print(field)
    try:
        print(getattr(data, field))
    except:
        print("N/A")
    print("-"*64)

In [104]:
file = '../../coarse2fine.git/data_model/django/train.pt'
fields = sorted(list(table.IO.TableDataset.get_fields().keys()))

data = torch.load(file)
data.fields = table.IO.TableDataset.get_fields()

print(type(data))

<class 'table.IO.TableDataset'>


In [105]:
i = random.randint(0, len(data.examples))

for f in fields:
    print_field(data.examples[i], f)

copy_to_ext
['if', 'prefix', 'is', 'None']
----------------------------------------------------------------
copy_to_tgt
['if', 'prefix', 'is', 'None']
----------------------------------------------------------------
indices
13752
----------------------------------------------------------------
lay
['if', 'NAME', 'is', 'None', ':']
----------------------------------------------------------------
lay_e
['if', 'NAME', 'is', 'None', ':']
----------------------------------------------------------------
lay_index
[0, 0, 1, 2, 3, 4]
----------------------------------------------------------------
lay_parent_index
[0, 0, 0, 0, 0, 0]
----------------------------------------------------------------
src
['if', 'prefix', 'is', 'None']
----------------------------------------------------------------
tgt
['<blank>', '<blank>', '<blank>', '<blank>', '<blank>']
----------------------------------------------------------------
tgt_copy_ext
['if', 'prefix', 'is', 'None', '<unk>']
------------------------

### Vocab

In [103]:
vocab = []

src_field = torchtext.data.Field(pad_token=PAD_WORD, include_lengths=True)

counter = Counter()
sources = [getattr(data, 'src')]

for data in sources:
    for x in data:
        counter.update(x)

counter.most_common(20)
# print(vocab)


AttributeError: 'generator' object has no attribute 'src'

In [44]:
def get_parent_index(tk_list):
    stack = [0]
    r_list = []
    
    for i, tk in enumerate(tk_list):
        print(i, tk)
        r_list.append(stack[-1])
        
        if tk.startswith('('):
            # +1: because the parent of the top level is 0
            stack.append(i + 1)
        elif tk == ')':
            stack.pop()
            
    # for EOS (</s>)
    r_list.append(0)
    return r_list


# get_parent_index('x = func ( a + func2 ( x + y ) )'.split())

0 x
1 =
2 func
3 (
4 a
5 +
6 func2
7 (
8 x
9 +
10 y
11 )
12 )


[0, 0, 0, 0, 4, 4, 4, 4, 8, 8, 8, 8, 4, 0]

In [51]:
def get_lay_index(lay_skip):
    # with a <s> token at the first position
    r_list = [0]
    k = 0
    for tk in lay_skip:
        if tk in (SKP_WORD, RIG_WORD):
            r_list.append(0)
        else:
            r_list.append(k)
            k += 1
    return r_list

get_lay_index('NAME = FUNC#1 ( NAME )'.split())

[0, 0, 1, 2, 3, 4, 5]